In [85]:
import pandas as pd
import numpy as np
import io
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
def resumen_valores_unicos_limitado(df, max_valores=20):
    columnas = df.columns
    valores_unicos = [df[col].unique()[:max_valores].tolist() for col in columnas]

    df_resumen = pd.DataFrame({
        'columna': columnas,
        'valores_unicos': valores_unicos
    })

    return df_resumen

## Rutas

### Proyectos

In [3]:
path_proyectos = 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/DS07_PROYECTOS_INVERSION.csv'
cols_proyectos= ["COD_SNIP", "NOMBRE_INVERSION", "CENT_RUC","CENT_NOMBRE", "NOMBRE_DEPARTAMENTO", 
            "NOMBRE_PROVINCIA", "NOMBRE_DISTRITO", "MONTO_VIABLE", "COSTO_ACTUALIZADO",
            "ESTADO", "FECHA_VIAB_APROB", "FEC_CIERRE", "MOD_EJECUCION", "MONTO_PIM",
            "MONTO_DEVENGADO", "MONTO_GIRADO", "MONTO_PAGADO", "AVANCE_FISICO_REAL",
            "AVANCE_INVERSION", "EJEC_FIS_PROGRAM_INICIO", "EJEC_FIS_REAL_INICIO", 
            "EJEC_FIS_PROGRAM_FINAL", "EJEC_FIS_REAL_FINAL", "TIPO_INVERSION",
            "DES_FUNCION", "SEC_EJEC", "MONTO_DEV_ACUMULADO", "ULT_PERIODO_DEV", "FASE_INVERSION"]


df_proyectos = pd.read_csv(path_proyectos,usecols=cols_proyectos,
                           dtype={'COD_SNIP':str,'CENT_RUC':str, 'ULT_PERIODO_DEV':str,
                            'SEC_EJEC':str})

C:\Users\soporte\AppData\Local\Temp\ipykernel_23336\3417103015.py:11: DtypeWarning: Columns (22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proyectos = pd.read_csv(path_proyectos,usecols=cols_proyectos,


In [27]:

def generar_df_training(df):
    # Eliminación de columnas innecesarias
    df_proyectos_01 = df.drop(columns=["NOMBRE_INVERSION", "CENT_NOMBRE", 'NOMBRE_DEPARTAMENTO',
                                       'NOMBRE_PROVINCIA', 'NOMBRE_DISTRITO', 'TIPO_INVERSION',
                                       'MONTO_VIABLE', 'COSTO_ACTUALIZADO', 'MONTO_PIM', 
                                       'MONTO_DEVENGADO', 'MONTO_GIRADO', 'MONTO_PAGADO',
                                       'MONTO_DEV_ACUMULADO', 'ULT_PERIODO_DEV'])


    # Conversión de fechas
    fechas = ['FECHA_VIAB_APROB', 'EJEC_FIS_PROGRAM_INICIO', 'EJEC_FIS_PROGRAM_FINAL', 
              'EJEC_FIS_REAL_FINAL', 'EJEC_FIS_REAL_INICIO']
    for fecha in fechas:
        df_proyectos_01[fecha] = pd.to_datetime(df_proyectos_01[fecha])

    # Eliminar duplicados y la columna FASE_INVERSION
    df_proyectos_01.drop(columns='FASE_INVERSION', inplace=True)
    df_proyectos_01.drop_duplicates('COD_SNIP',inplace=True)

    return df_proyectos_01


In [28]:
df_proyectos_01= generar_df_training(df_proyectos)

In [29]:
df_proyectos_01.shape

(456727, 14)

In [30]:
df_proyectos_01.COD_SNIP.nunique()

456727

In [25]:
df_proyectos_01.head()

,COD_SNIP,CENT_RUC,ESTADO,FECHA_VIAB_APROB,FEC_CIERRE,MOD_EJECUCION,AVANCE_FISICO_REAL,AVANCE_INVERSION,EJEC_FIS_PROGRAM_INICIO,EJEC_FIS_REAL_INICIO,EJEC_FIS_PROGRAM_FINAL,EJEC_FIS_REAL_FINAL,DES_FUNCION,SEC_EJEC
0,2383209,20147796634,ACTIVO,2017-09-27,NaN,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,91.74,87.84,2019-06-03,2019-08-23,2019-09-01,2020-10-09,EDUCACIÓN,109
40,2384501,20137921601,ACTIVO,2017-09-26,NaN,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,NaN,NaN,NaT,NaT,NaT,NaT,PESCA,192
42,2382775,20154477374,CERRADO,2017-09-27,24-MAR-22,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,93.47,93.62,2017-12-18,2018-10-18,2018-03-17,2024-07-20,TRANSPORTE,301529
45,2351502,20172356720,CERRADO,2017-10-03,01-JAN-20,ADMINISTRACIÓN INDIRECTA - POR CONTRATA,100.00,NaN,2017-05-11,2018-01-25,2017-07-10,2017-06-28,EDUCACIÓN,104
47,2396945,20201975361,DESACTIVADO PERMANENTE,2017-10-06,NaN,ADMINISTRACIÓN DIRECTA,NaN,NaN,NaT,NaT,NaT,NaT,SANEAMIENTO,301734


### Consulta avanzada

In [7]:
path_consulta_avanzada = 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/consulta_avanzada.csv'
df_consulta_avanzada = pd.read_csv(path_consulta_avanzada,
                                   dtype={'Código único de inversión':str,
                                        'Código SNIP':str,'Ubigeo':str})



In [ ]:
df_consulta_avanzada['Fecha de registro'] = pd.to_datetime(df_consulta_avanzada['Fecha de registro'])
df_consulta_avanzada['Fecha de viabilidad'] = pd.to_datetime(df_consulta_avanzada['Fecha de viabilidad'])

# Agrupar por Código único de inversión y Código SNIP, y calcular las nuevas columnas
df_agrupado_consulta = df_consulta_avanzada.groupby(['Código único de inversión', 'Código SNIP']).agg(
    monto_aprobado=pd.NamedAgg(column='Monto viable', aggfunc='mean'),
    fecha_inicio=pd.NamedAgg(column='Fecha de registro', aggfunc='first'),
    fecha_aprobado=pd.NamedAgg(column='Fecha de viabilidad', aggfunc='first'),
    costo_final=pd.NamedAgg(column='Costo actualizado', aggfunc='mean'),
    saldo_financiar=pd.NamedAgg(column='Saldo por financiar', aggfunc='mean'),
    nro_lugares=pd.NamedAgg(column='Ubigeo', aggfunc=lambda x: x.nunique()),
    ubigeo=pd.NamedAgg(column='Ubigeo', aggfunc=lambda x: x.iloc[0])
).reset_index()

# Cambiar los nombres de las columnas
df_agrupado_consulta.rename(columns={'Código único de inversión': 'CUI', 
                            'Código SNIP': 'COD_SNIP'}, inplace=True)

df_agrupado_consulta['devengado']= df_agrupado_consulta.costo_final-df_agrupado_consulta.saldo_financiar

### data inei

In [9]:
path_inei_dist = 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/ubigeo_distrito.csv'
df_inei_dist = pd.read_csv(path_inei_dist,usecols=['inei','macroregion_inei','pob_densidad_2020', 'altitude', 'latitude',
       'longitude', 'indice_vulnerabilidad_alimentaria', 'idh_2019',
       'pct_pobreza_total', 'pct_pobreza_extrema'],
                                   dtype={'inei':str})
df_inei_dist.rename(columns={'latitude':'latitude_proy','longitude':'longitude_proy'},inplace=True)
df_inei_dist['ubi_geo'] = df_inei_dist['inei'].str[:2]

path_inei_dep = 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/ubigeo_departamento.csv'
df_inei_dep = pd.read_csv(path_inei_dep,usecols=['inei', 'longitude', 'latitude'],dtype={'inei':str})
df_inei_dep['ubi_geo'] = df_inei_dep['inei'].str[:2]


In [10]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Convertir a coordenadas cartesianas
    x1, y1, z1 = np.cos(lat1) * np.cos(lon1), np.cos(lat1) * np.sin(lon1), np.sin(lat1)
    x2, y2, z2 = np.cos(lat2) * np.cos(lon2), np.cos(lat2) * np.sin(lon2), np.sin(lat2)

    # Calcular la distancia euclidiana
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

    # Convertir la distancia de unidades terrestres a kilómetros
    # El radio medio de la Tierra es aproximadamente 6371 kilómetros
    distance_km = distance * 6371

    return distance_km

### Educación

In [11]:
path__educacion= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/educacion.csv'
df_educacion = pd.read_csv(path__educacion, sep=';', dtype={'codgeo':str})

In [225]:
df_educacion.head()

,codgeo,ausentismo,analfabetismo,secundaria
0,010101,0.076698,0.033878,9070
1,010102,0.098039,0.147059,62
2,010103,0.101754,0.129887,257
3,010104,0.052632,0.063136,150
4,010105,0.137931,0.138702,78


### Entidades

In [13]:
path_entidades= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/master_entidad_seace.csv'
df_entidades = pd.read_csv(path_entidades, dtype={'CENT_RUC':str})
df_entidades.drop(columns=['STD_DIF_PUB_ADJU', 'STD_PORC_ADJ_PROVEEDOR','CENT_NOMBRE', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_PROVINCIA',
       'NOMBRE_DISTRITO'], inplace=True)


In [14]:
df_entidades.isnull().sum()

CENT_RUC                                       0
ANO_ADJ                                        0
CANTI_EXP                                      0
TOTAL_ADJ_PROVEEDOR_SOLES                      0
NRO_PROVEEDORES                                0
PORC_CONTRATACIÓN_DIRECTA                      0
PORC_ADJUDICACIÓN_SIMPLIFICADA                 0
PORC_SUBASTA_INVERSA_ELECTRÓNICA               0
PORC_LICITACIÓN_PÚBLICA                        0
PORC_REGÍMEN_ESPECIAL                          0
PORC_CONCURSO_PÚBLICO                          0
PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN    0
PORC_PROVEEDOR_CONS                            0
dtype: int64

In [15]:
df_entidades.head()

,CENT_RUC,ANO_ADJ,CANTI_EXP,TOTAL_ADJ_PROVEEDOR_SOLES,NRO_PROVEEDORES,PORC_CONTRATACIÓN_DIRECTA,PORC_ADJUDICACIÓN_SIMPLIFICADA,PORC_SUBASTA_INVERSA_ELECTRÓNICA,PORC_LICITACIÓN_PÚBLICA,PORC_REGÍMEN_ESPECIAL,PORC_CONCURSO_PÚBLICO,PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN,PORC_PROVEEDOR_CONS
0,20100003199,2020.0,19,8.906413e+06,19,5.263158,63.157895,0.0,5.263158,0.0,26.315789,0.0,10.526316
1,20100003199,2021.0,19,7.326682e+06,18,5.263158,63.157895,0.0,5.263158,0.0,26.315789,0.0,0.000000
2,20100003199,2022.0,19,6.527610e+06,22,0.000000,110.526316,0.0,0.000000,0.0,15.789474,0.0,27.272727
3,20100003351,2020.0,32,1.459707e+07,36,15.625000,93.750000,0.0,3.125000,0.0,18.750000,0.0,38.888889
4,20100003351,2021.0,44,2.028464e+07,42,2.272727,81.818182,0.0,4.545455,0.0,25.000000,0.0,16.666667


### Proveedor inversión

In [16]:
path_proveedores_inversión= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/df_proveedores_1_filtrado.csv'

df_proveedores_inversión = pd.read_csv(path_proveedores_inversión, dtype={'PROD_PROY':str, 'RUC':str})
df_proveedores_inversión=df_proveedores_inversión.loc[:,['PROD_PROY','RUC']]
df_proveedores_inversión.drop_duplicates(inplace=True)

### master proveedor

path_master_proveedor= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/master_proveedor.csv'

df_master_proveedor = pd.read_csv(path_master_proveedor)

In [17]:
### master proveedor

path_master_proveedor= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/master_proveedor.csv'

df_master_proveedor = pd.read_csv(path_master_proveedor)

C:\Users\soporte\AppData\Local\Temp\ipykernel_23336\2373123324.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_master_proveedor = pd.read_csv(path_master_proveedor)


In [19]:
df_master_proveedor.head(10)

,RUC_PROVEEDOR,ACT_PRIMARIA,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,DEPARTAMENTO,PROVINCIA,DISTRITO,LATITUDE,LONGITUDE,NRO_EXP,TOTAL_ADJ_PROVEEDOR
0,10100275560,NaN,ACTIVO,NO HABIDO,UCAYALI,CORONEL PORTILLO,CALLERIA,-8.368056,-74.543333,NaN,NaN
1,10100793370,NaN,ACTIVO,HABIDO,TUMBES,TUMBES,TUMBES,-3.571111,-80.459167,NaN,NaN
2,10100970330,OTRAS ACTIVIDADES ESPECIALIZADAS DE LA CONSTRU...,ACTIVO,HABIDO,LIMA,LIMA,SAN JUAN DE LURIGANCHO,-12.029722,-77.010000,NaN,NaN
3,10101299540,NaN,ACTIVO,HABIDO,TACNA,TACNA,TACNA,-18.001944,-70.251944,NaN,NaN
4,10101679950,NaN,ACTIVO,HABIDO,TACNA,TACNA,ALTO DE LA ALIANZA,-17.993056,-70.247778,NaN,NaN
5,10101753220,NaN,ACTIVO,HABIDO,TACNA,TACNA,TACNA,-18.001944,-70.251944,NaN,NaN
6,10101849660,"OTRAS ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y...",SUSPENSION TEMPORAL,HABIDO,SAN MARTIN,MOYOBAMBA,MOYOBAMBA,-6.034722,-76.974167,NaN,NaN
7,10101861940,NaN,ACTIVO,NO HABIDO,SAN MARTIN,MOYOBAMBA,CALZADA,-6.030278,-77.066667,NaN,NaN
8,10101879470,ENSEÑANZA SECUNDARIA DE FORMACIÓN GENERAL,ACTIVO,HABIDO,SAN MARTIN,MOYOBAMBA,MOYOBAMBA,-6.034722,-76.974167,NaN,NaN
9,10102026610,ACTIVIDADES DE RESTAURANTES Y DE SERVICIO MÓVI...,ACTIVO,HABIDO,SAN MARTIN,SAN MARTIN,MORALES,-6.479167,-76.383056,NaN,NaN


In [ ]:
df_master_proveedor

### combinación

In [35]:
df_proyectos_final= df_proyectos_01.merge(df_agrupado_consulta, how='left', on='COD_SNIP')
# Seleccionar las columnas requeridas y realizar las operaciones indicadas
df_proyectos_indicadores = df_proyectos_final[['COD_SNIP', 'CENT_RUC', 'MOD_EJECUCION', 
                                               'DES_FUNCION', 'SEC_EJEC', 'nro_lugares', 
                                               'ubigeo', 'CUI', 'monto_aprobado', 'costo_final', 
                                               'devengado', 'EJEC_FIS_PROGRAM_INICIO', 
                                               'EJEC_FIS_REAL_INICIO', 'EJEC_FIS_PROGRAM_FINAL', 
                                               'EJEC_FIS_REAL_FINAL', 'FECHA_VIAB_APROB']].copy()


# Calcular porcentajes y diferencias de tiempo
df_proyectos_indicadores['porc_incremento_monto'] = (df_proyectos_indicadores['costo_final'] - df_proyectos_indicadores['monto_aprobado']) / df_proyectos_indicadores['monto_aprobado']
df_proyectos_indicadores['porc_gastado'] = (df_proyectos_indicadores['costo_final'] - df_proyectos_indicadores['devengado']) / df_proyectos_indicadores['costo_final']

# Conversión de fechas a datetime si no lo están
fechas = ['EJEC_FIS_PROGRAM_INICIO', 'EJEC_FIS_REAL_INICIO', 'EJEC_FIS_PROGRAM_FINAL', 
          'EJEC_FIS_REAL_FINAL', 'FECHA_VIAB_APROB']
for fecha in fechas:
    df_proyectos_indicadores[fecha] = pd.to_datetime(df_proyectos_indicadores[fecha])

# Calcular diferencias de tiempo en meses
df_proyectos_indicadores['tiempo_programado_meses'] = (df_proyectos_indicadores['EJEC_FIS_PROGRAM_FINAL'] - df_proyectos_indicadores['EJEC_FIS_PROGRAM_INICIO']).dt.days / 30
df_proyectos_indicadores['tiempo_real_meses'] = (df_proyectos_indicadores['EJEC_FIS_REAL_FINAL'] - df_proyectos_indicadores['EJEC_FIS_REAL_INICIO']).dt.days / 30
df_proyectos_indicadores['tiempo_demora_inicio_meses'] = (df_proyectos_indicadores['EJEC_FIS_REAL_INICIO'] - df_proyectos_indicadores['EJEC_FIS_PROGRAM_INICIO']).dt.days / 30
df_proyectos_indicadores['tiempo_demora_aprobado_meses'] = (df_proyectos_indicadores['EJEC_FIS_REAL_INICIO'] - df_proyectos_indicadores['FECHA_VIAB_APROB']).dt.days / 30

# Calcular porcentaje de tiempo adicional
df_proyectos_indicadores['porc_tiempo_adicional'] = (abs(df_proyectos_indicadores['tiempo_real_meses'] - df_proyectos_indicadores['tiempo_programado_meses'])/ df_proyectos_indicadores['tiempo_programado_meses'])*100

# Estandarizar valores de MOD_EJECUCION usando contains
df_proyectos_indicadores['MOD_EJECUCION'] = df_proyectos_indicadores['MOD_EJECUCION'].str.contains('INDIRECTA').replace({True: 'INDIRECTA', False: 'DIRECTA'})

# Verificar y corregir posibles divisiones por cero o valores NaN
df_proyectos_indicadores.replace([np.inf, -np.inf], 0, inplace=True)
df_proyectos_indicadores= df_proyectos_indicadores.dropna()
df_proyectos_indicadores.drop(columns=['EJEC_FIS_PROGRAM_INICIO', 'EJEC_FIS_REAL_INICIO',
       'EJEC_FIS_PROGRAM_FINAL', 'EJEC_FIS_REAL_FINAL'], inplace=True)


df_proyectos_indicadores= df_proyectos_indicadores.merge(df_inei_dist, how='left', left_on='ubigeo', right_on='inei')
df_proyectos_indicadores= df_proyectos_indicadores.merge(df_inei_dep, how='left', on='ubi_geo')

df_proyectos_indicadores['distancia'] = df_proyectos_indicadores.apply(lambda row: euclidean_distance(row['latitude_proy'], row['longitude_proy'], 
                                                         row['latitude'], row['longitude']), axis=1)

cols_to_drop=['inei_x', 'macroregion_inei',
       'pob_densidad_2020', 'altitude', 
       'indice_vulnerabilidad_alimentaria', 'idh_2019', 'pct_pobreza_total',
       'pct_pobreza_extrema','ubigeo', 'ubi_geo',
       'inei_y', 'latitude', 'longitude',]
df_proyectos_indicadores.drop(columns=cols_to_drop, inplace=True)

#################PROVEEDORES

#df_proyectos_indicadores=df_proyectos_indicadores.merge(df_proveedores_inversión, how='inner', left_on='CUI', right_on='PROD_PROY')

################# Definir el umbral 8UIT
umbral = 39600

# Crear la columna flag_8_uit
df_proyectos_indicadores['flag_8_uit'] = (df_proyectos_indicadores['monto_aprobado'] >= umbral).astype(int)

# Crear la columna cercania_8_uit
df_proyectos_indicadores['cercania_8_uit'] = (umbral - df_proyectos_indicadores['monto_aprobado'])/umbral
df_proyectos_indicadores['cercania_8_uit'] = df_proyectos_indicadores['cercania_8_uit'].apply(lambda x: x if x > 0 else 1)
df_proyectos_indicadores['ANO_ADJ'] = df_proyectos_indicadores['FECHA_VIAB_APROB'].dt.year

##################### AGREGANDO ENTIDADES###########
df_proyectos_indicadores= df_proyectos_indicadores.merge(df_entidades, how='inner', on=['CENT_RUC','ANO_ADJ' ])



In [38]:
df_proyectos_indicadores.columns

Index(['COD_SNIP', 'CENT_RUC', 'MOD_EJECUCION', 'DES_FUNCION', 'SEC_EJEC',
       'nro_lugares', 'CUI', 'monto_aprobado', 'costo_final', 'devengado',
       'FECHA_VIAB_APROB', 'porc_incremento_monto', 'porc_gastado',
       'tiempo_programado_meses', 'tiempo_real_meses',
       'tiempo_demora_inicio_meses', 'tiempo_demora_aprobado_meses',
       'porc_tiempo_adicional', 'latitude_proy', 'longitude_proy', 'distancia',
       'flag_8_uit', 'cercania_8_uit', 'ANO_ADJ', 'CANTI_EXP',
       'TOTAL_ADJ_PROVEEDOR_SOLES', 'NRO_PROVEEDORES',
       'PORC_CONTRATACIÓN_DIRECTA', 'PORC_ADJUDICACIÓN_SIMPLIFICADA',
       'PORC_SUBASTA_INVERSA_ELECTRÓNICA', 'PORC_LICITACIÓN_PÚBLICA',
       'PORC_REGÍMEN_ESPECIAL', 'PORC_CONCURSO_PÚBLICO',
       'PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN', 'PORC_PROVEEDOR_CONS'],
      dtype='object')

In [49]:
path_proveedores_inversión= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/df_proveedores_1_filtrado.csv'

df_proveedores_inversión = pd.read_csv(path_proveedores_inversión, dtype={'PROD_PROY':str, 'RUC':str})
df_proveedores_inversión=df_proveedores_inversión.loc[:,['PROD_PROY','RUC']]
df_proveedores_inversión.rename(columns={'PROD_PROY':'CUI'},inplace=True)
df_proveedores_inversión.drop_duplicates(inplace=True)

### master proveedor

path_master_proveedor= 'C:/Users/soporte/Documents/cdr/data/07_DS_PROYECTOS DE INVERSION/master_proveedor.csv'


df_master_proveedor = pd.read_csv(path_master_proveedor, dtype={'RUC_PROVEEDOR':str})
df_master_proveedor=df_master_proveedor.loc[:,['RUC_PROVEEDOR','ESTADO_PROVEEDOR','CONDICION_PROVEEDOR','LATITUDE','LONGITUDE']]
df_master_proveedor.rename(columns={'RUC_PROVEEDOR':'RUC'},inplace=True)

df_proveedor= df_proveedores_inversión.merge(df_proyectos_indicadores, how='inner', on='CUI')
df_proveedor=df_proveedor.loc[:, ['CUI','RUC', 'latitude_proy','longitude_proy']].merge(df_master_proveedor, how='inner', on='RUC')
df_proveedor['distancia_proveedor'] = df_proveedor.apply(lambda row: euclidean_distance(row['latitude_proy'], row['longitude_proy'], 
                                                         row['LATITUDE'], row['LONGITUDE']), axis=1)
umbral_km=100
df_proveedor['flag_distancia'] = (df_proveedor['distancia_proveedor'] >= umbral_km).astype(int)
df_proveedor=df_proveedor.loc[:,['CUI','RUC','ESTADO_PROVEEDOR','CONDICION_PROVEEDOR','distancia_proveedor','flag_distancia']]

df_proveedor = df_proveedor.groupby('CUI').agg(
    nro_proveedores=('RUC', 'nunique'),
    porc_sin_actividad=('ESTADO_PROVEEDOR', lambda x: (x != 'ACTIVO').mean() * 100),
    porc_sin_no_habido=('CONDICION_PROVEEDOR', lambda x: (x != 'HABIDO').mean() * 100),
    promedio_distancias=('distancia_proveedor', 'mean'),
    porc_proveedor_cercano=('flag_distancia', lambda x: (x == 0).mean() * 100)
)
df_proveedor.reset_index(inplace=True)


In [86]:
df_master.columns

Index(['COD_SNIP', 'CENT_RUC', 'MOD_EJECUCION', 'DES_FUNCION', 'SEC_EJEC',
       'nro_lugares', 'CUI', 'monto_aprobado', 'costo_final', 'devengado',
       'FECHA_VIAB_APROB', 'porc_incremento_monto', 'porc_gastado',
       'tiempo_programado_meses', 'tiempo_real_meses',
       'tiempo_demora_inicio_meses', 'tiempo_demora_aprobado_meses',
       'porc_tiempo_adicional', 'latitude_proy', 'longitude_proy', 'distancia',
       'flag_8_uit', 'cercania_8_uit', 'ANO_ADJ', 'CANTI_EXP',
       'TOTAL_ADJ_PROVEEDOR_SOLES', 'NRO_PROVEEDORES',
       'PORC_CONTRATACIÓN_DIRECTA', 'PORC_ADJUDICACIÓN_SIMPLIFICADA',
       'PORC_SUBASTA_INVERSA_ELECTRÓNICA', 'PORC_LICITACIÓN_PÚBLICA',
       'PORC_REGÍMEN_ESPECIAL', 'PORC_CONCURSO_PÚBLICO',
       'PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN', 'PORC_PROVEEDOR_CONS',
       'nro_proveedores', 'porc_sin_actividad', 'porc_sin_no_habido',
       'promedio_distancias', 'porc_proveedor_cercano'],
      dtype='object')

In [87]:
df_master= df_proyectos_indicadores.merge(df_proveedor, how='inner', on='CUI')

categorical_vars = ['MOD_EJECUCION', 'DES_FUNCION']  # Actualiza con los nombres de tus columnas

# Inicializando OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')  # 'drop' es opcional, elimina la primera categoría para evitar la multicolinealidad

# Aplicar OneHotEncoder a las variables categóricas
encoded_vars = encoder.fit_transform(df_master[categorical_vars])

# Crear un DataFrame con las variables codificadas
encoded_df = pd.DataFrame(encoded_vars, columns=encoder.get_feature_names(categorical_vars))

# Unir este DataFrame codificado con el DataFrame original
# Nota: Es importante resetear el índice si se han realizado operaciones de filtrado o muestreo en el DataFrame original
df_master.reset_index(drop=True, inplace=True)
df_encoded = pd.concat([df_master, encoded_df], axis=1)

# Opcional: Eliminar las columnas originales después de codificarlas
df_encoded.drop(categorical_vars, axis=1, inplace=True)

c:\Users\soporte\anaconda3\envs\data_science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [89]:
df_encoded.shape

(22236, 60)

In [94]:
columns_train=['COD_SNIP', 'CENT_RUC', 
       'nro_lugares', 'CUI', 'monto_aprobado', 
       'tiempo_programado_meses',
       'distancia',
       'flag_8_uit', 'cercania_8_uit', 'ANO_ADJ', 'CANTI_EXP',
       'TOTAL_ADJ_PROVEEDOR_SOLES', 'NRO_PROVEEDORES',
       'PORC_CONTRATACIÓN_DIRECTA', 'PORC_ADJUDICACIÓN_SIMPLIFICADA',
       'PORC_SUBASTA_INVERSA_ELECTRÓNICA', 'PORC_LICITACIÓN_PÚBLICA',
       'PORC_REGÍMEN_ESPECIAL', 'PORC_CONCURSO_PÚBLICO',
       'PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN', 'PORC_PROVEEDOR_CONS',
       'MOD_EJECUCION_INDIRECTA', 'DES_FUNCION_AMBIENTE',
       'DES_FUNCION_COMERCIO', 'DES_FUNCION_COMUNICACIONES',
       'DES_FUNCION_CULTURA Y DEPORTE',
       'DES_FUNCION_DEFENSA Y SEGURIDAD NACIONAL', 'DES_FUNCION_EDUCACIÓN',
       'DES_FUNCION_ENERGÍA', 'DES_FUNCION_INDUSTRIA', 'DES_FUNCION_JUSTICIA',
       'DES_FUNCION_ORDEN PÚBLICO Y SEGURIDAD', 'DES_FUNCION_OTROS',
       'DES_FUNCION_PESCA',
       'DES_FUNCION_PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTINGENCIA',
       'DES_FUNCION_PREVISIÓN SOCIAL', 'DES_FUNCION_PROTECCIÓN SOCIAL',
       'DES_FUNCION_SALUD', 'DES_FUNCION_SANEAMIENTO', 'DES_FUNCION_TRABAJO',
       'DES_FUNCION_TRANSPORTE', 'DES_FUNCION_TURISMO',
       'DES_FUNCION_VIVIENDA Y DESARROLLO URBANO']


columns_flag=['COD_SNIP', 
        'porc_incremento_monto', 'porc_gastado',
       'porc_tiempo_adicional',  'ANO_ADJ',
       'nro_proveedores', 'porc_sin_actividad', 'porc_sin_no_habido',
       'promedio_distancias', 'porc_proveedor_cercano']

In [95]:
df_inputs=df_encoded.loc[:,columns_train]

In [96]:
df_flag= df_encoded.loc[:,columns_flag]
df_flag.fillna(0, inplace=True)

# Escalar los datos
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_flag[['porc_incremento_monto', 'porc_gastado', 
                                                'porc_tiempo_adicional', 'nro_proveedores', 
                                                'porc_sin_actividad', 'porc_sin_no_habido', 
                                                'promedio_distancias', 'porc_proveedor_cercano']])

# Ponderaciones asignadas a cada variable
weights = np.array([0.20, 0.15, 0.20, 0.10, 0.15, 0.10, 0.05, 0.05])

# Calcular el índice como suma ponderada
risk_index = np.dot(scaled_features, weights)

# Añadir el índice al DataFrame
df_flag['risk_index'] = risk_index

# Clasificar usando percentiles
threshold = np.percentile(risk_index, 75)  # Puedes ajustar este valor según necesidades
df_flag['risk_category'] = np.where(df_flag['risk_index'] >= threshold, 1, 0)
df_flag=df_flag.loc[:, ['COD_SNIP',  'risk_category']]

### Generando master training y predict

In [112]:
df_master_data= df_inputs.merge(df_flag, how='inner', on='COD_SNIP')

def clean_column_names(df):
    df.columns = [col.replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace(':', '').replace(',', '') for col in df.columns]
    return df

# Limpiar los nombres de las columnas de X_train y X_test
df_master_data = clean_column_names(df_master_data)

df_master_train= df_master_data.loc[df_master_data.ANO_ADJ!=2022]
df_master_test= df_master_data.loc[df_master_data.ANO_ADJ==2022]

In [108]:
X_train.shape

(16727, 39)

In [109]:
X_test.shape

(5509, 41)

In [111]:
X_train.columns

Index(['nro_lugares', 'monto_aprobado', 'tiempo_programado_meses', 'distancia',
       'flag_8_uit', 'cercania_8_uit', 'CANTI_EXP',
       'TOTAL_ADJ_PROVEEDOR_SOLES', 'NRO_PROVEEDORES',
       'PORC_CONTRATACIÓN_DIRECTA', 'PORC_ADJUDICACIÓN_SIMPLIFICADA',
       'PORC_SUBASTA_INVERSA_ELECTRÓNICA', 'PORC_LICITACIÓN_PÚBLICA',
       'PORC_REGÍMEN_ESPECIAL', 'PORC_CONCURSO_PÚBLICO',
       'PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN', 'PORC_PROVEEDOR_CONS',
       'MOD_EJECUCION_INDIRECTA', 'DES_FUNCION_AMBIENTE',
       'DES_FUNCION_COMERCIO', 'DES_FUNCION_COMUNICACIONES',
       'DES_FUNCION_CULTURA Y DEPORTE',
       'DES_FUNCION_DEFENSA Y SEGURIDAD NACIONAL', 'DES_FUNCION_EDUCACIÓN',
       'DES_FUNCION_ENERGÍA', 'DES_FUNCION_INDUSTRIA', 'DES_FUNCION_JUSTICIA',
       'DES_FUNCION_ORDEN PÚBLICO Y SEGURIDAD', 'DES_FUNCION_OTROS',
       'DES_FUNCION_PESCA',
       'DES_FUNCION_PLANEAMIENTO GESTIÓN Y RESERVA DE CONTINGENCIA',
       'DES_FUNCION_PREVISIÓN SOCIAL', 'DES_FUNCION_PROTECC

In [110]:
X_test.columns

Index(['nro_lugares', 'monto_aprobado', 'tiempo_programado_meses', 'distancia',
       'flag_8_uit', 'cercania_8_uit', 'CANTI_EXP',
       'TOTAL_ADJ_PROVEEDOR_SOLES', 'NRO_PROVEEDORES',
       'PORC_CONTRATACIÓN_DIRECTA', 'PORC_ADJUDICACIÓN_SIMPLIFICADA',
       'PORC_SUBASTA_INVERSA_ELECTRÓNICA', 'PORC_LICITACIÓN_PÚBLICA',
       'PORC_REGÍMEN_ESPECIAL', 'PORC_CONCURSO_PÚBLICO',
       'PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN', 'PORC_PROVEEDOR_CONS',
       'MOD_EJECUCION_INDIRECTA', 'DES_FUNCION_AMBIENTE',
       'DES_FUNCION_COMERCIO', 'DES_FUNCION_COMUNICACIONES',
       'DES_FUNCION_CULTURA Y DEPORTE',
       'DES_FUNCION_DEFENSA Y SEGURIDAD NACIONAL', 'DES_FUNCION_EDUCACIÓN',
       'DES_FUNCION_ENERGÍA', 'DES_FUNCION_INDUSTRIA', 'DES_FUNCION_JUSTICIA',
       'DES_FUNCION_ORDEN PÚBLICO Y SEGURIDAD', 'DES_FUNCION_OTROS',
       'DES_FUNCION_PESCA',
       'DES_FUNCION_PLANEAMIENTO GESTIÓN Y RESERVA DE CONTINGENCIA',
       'DES_FUNCION_PREVISIÓN SOCIAL', 'DES_FUNCION_PROTECC

In [113]:
# Supongamos que 'target' es tu columna objetivo
X_train = df_master_train.drop(['COD_SNIP', 'CENT_RUC','CUI', 'ANO_ADJ','risk_category'], axis=1)
y_train = df_master_train['risk_category']

X_test= df_master_test.drop(['COD_SNIP', 'CENT_RUC','CUI', 'ANO_ADJ','risk_category'], axis=1)
y_test = df_master_test['risk_category']

# Creando el modelo LightGBM
# Ajusta estos parámetros según tus necesidades y los resultados de cualquier ajuste de hiperparámetros que realices
model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    is_unbalance=True,  # Importante para tratar el desbalanceo
    n_estimators=100,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,
    random_state=42
)

# Entrenamiento del modelo
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

y_prob = model.predict_proba(X_test)[:, 1]

# Añadir las predicciones y probabilidades al DataFrame de test
# Asegúrate de que el índice del DataFrame de test coincida con el de X_test para una correcta alineación
#df_master_test = pd.DataFrame(X_test, columns=features_columns)  # Reemplaza 'features_columns' con los nombres de tus columnas de características
df_master_test['y_pred'] = y_pred
df_master_test['y_prob'] = y_prob

# Métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')

Accuracy: 0.7231802504991831
Precision: 0.303714493809177
Recall: 0.422920892494929
F1 Score: 0.3535396354387452
ROC AUC Score: 0.6057783768245152


C:\Users\soporte\AppData\Local\Temp\ipykernel_23336\2926998244.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_master_test['y_pred'] = y_pred
C:\Users\soporte\AppData\Local\Temp\ipykernel_23336\2926998244.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_master_test['y_prob'] = y_prob


In [81]:
df_master_test.head()

,COD_SNIP,CENT_RUC,MOD_EJECUCION,DES_FUNCION,nro_lugares,CUI,monto_aprobado,tiempo_programado_meses,distancia,flag_8_uit,...,NRO_PROVEEDORES,PORC_CONTRATACIÓN_DIRECTA,PORC_ADJUDICACIÓN_SIMPLIFICADA,PORC_SUBASTA_INVERSA_ELECTRÓNICA,PORC_LICITACIÓN_PÚBLICA,PORC_REGÍMEN_ESPECIAL,PORC_CONCURSO_PÚBLICO,PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN,PORC_PROVEEDOR_CONS,risk_category
152,2513642,20393066386,INDIRECTA,EDUCACIÓN,15.0,2513642,3333371.61,8.066667,3.589327,1,...,168,6.666667,134.444444,3.333333,45.555556,0.0,16.666667,0.0,56.547619,0
153,2514080,20393066386,INDIRECTA,EDUCACIÓN,17.0,2514080,16652881.74,8.400000,55.308545,1,...,168,6.666667,134.444444,3.333333,45.555556,0.0,16.666667,0.0,56.547619,1
154,2514399,20393066386,DIRECTA,EDUCACIÓN,17.0,2514399,3629682.61,9.033333,448.866480,1,...,168,6.666667,134.444444,3.333333,45.555556,0.0,16.666667,0.0,56.547619,0
155,2548173,20393066386,DIRECTA,ORDEN PÚBLICO Y SEGURIDAD,7.0,2548173,3744862.90,3.066667,55.308545,1,...,168,6.666667,134.444444,3.333333,45.555556,0.0,16.666667,0.0,56.547619,0
156,2548175,20393066386,DIRECTA,ORDEN PÚBLICO Y SEGURIDAD,10.0,2548175,9807791.29,4.066667,75.651974,1,...,168,6.666667,134.444444,3.333333,45.555556,0.0,16.666667,0.0,56.547619,0


In [79]:
df_inputs.dtypes

COD_SNIP                                        object
CENT_RUC                                        object
MOD_EJECUCION                                   object
DES_FUNCION                                     object
nro_lugares                                    float64
CUI                                             object
monto_aprobado                                 float64
tiempo_programado_meses                        float64
distancia                                      float64
flag_8_uit                                       int32
cercania_8_uit                                 float64
ANO_ADJ                                          int64
CANTI_EXP                                        int64
TOTAL_ADJ_PROVEEDOR_SOLES                      float64
NRO_PROVEEDORES                                  int64
PORC_CONTRATACIÓN_DIRECTA                      float64
PORC_ADJUDICACIÓN_SIMPLIFICADA                 float64
PORC_SUBASTA_INVERSA_ELECTRÓNICA               float64
PORC_LICIT

In [78]:
df_master_training.ANO_ADJ.value_counts()

2020    8623
2021    8065
2022    5509
2019      39
Name: ANO_ADJ, dtype: int64

In [69]:
df_flag.head()

,COD_SNIP,porc_incremento_monto,porc_gastado,porc_tiempo_adicional,ANO_ADJ,nro_proveedores,porc_sin_actividad,porc_sin_no_habido,promedio_distancias,porc_proveedor_cercano,risk_index,risk_category
0,2535995,0.068550,0.023734,32.608696,2021,4,0.0,0.0,14.954779,100.0,-0.256849,0
1,2509842,0.460870,0.004433,97.802198,2021,3,0.0,0.0,21.216312,100.0,-0.164727,0
2,2526990,-0.000003,0.000000,4.444444,2021,3,0.0,0.0,21.216312,100.0,-0.292511,0
3,2527961,-0.000520,0.000000,29.032258,2021,3,0.0,0.0,21.216312,100.0,-0.288051,0
4,2535979,0.532620,0.022061,432.352941,2021,15,0.0,0.0,4.424103,100.0,-0.034548,0


'porc_incremento_monto' (0.20): Un aumento significativo en el monto presupuestado puede ser indicativo de mala gestión o corrupción. Sin embargo, también puede deberse a cambios legítimos en el alcance del proyecto.

'porc_gastado' (0.15): Un alto porcentaje de gasto puede indicar eficiencia, pero debe ser equilibrado con otros indicadores para asegurar que el gasto es justificado y no resultado de sobrecostos.

'porc_tiempo_adicional' (0.20): Los retrasos significativos en los proyectos pueden ser un signo de problemas de gestión o corrupción, especialmente si se combinan con sobrecostos.

'nro_proveedores' (0.10): Un número más alto de proveedores puede indicar una diversificación saludable, pero un número demasiado bajo o alto puede ser problemático. La ponderación es neutral debido a su naturaleza ambigua.

'porc_sin_actividad' (0.15): Un alto porcentaje de proveedores sin actividad podría indicar prácticas corruptas, como la inclusión de empresas fantasma.

'porc_sin_no_habido' (0.10): Similar a 'porc_sin_actividad', indica posibles irregularidades, aunque podría ser menos directamente relacionado con la corrupción.

'promedio_distancias' (0.05): Mientras que las distancias cortas podrían indicar favoritismo local, este factor puede ser menos relevante para la corrupción en comparación con los factores financieros y operativos.

'porc_proveedor_cercano' (0.05): Un bajo porcentaje podría sugerir falta de transparencia en la selección de proveedores, pero este factor es menos crítico en comparación con los aspectos financieros y de cumplimiento del proyecto.

In [66]:
df_flag= df_master.loc[:,columns_flag]
df_flag.fillna(0, inplace=True)
# Excluir 'COD_SNIP' y ANIO
features = df_flag[['porc_incremento_monto', 'porc_gastado', 'porc_tiempo_adicional', 'nro_proveedores', 'porc_sin_actividad', 'porc_sin_no_habido', 'promedio_distancias', 'porc_proveedor_cercano']]
'''
# Invertir las variables negativas
features['porc_incremento_monto'] = 1 - features['porc_incremento_monto']
features['porc_tiempo_adicional'] = 1 - features['porc_tiempo_adicional']
features['porc_sin_actividad'] = 1 - features['porc_sin_actividad']
features['porc_sin_no_habido'] = 1 - features['porc_sin_no_habido']
features['promedio_distancias'] = 1 / (1 + features['promedio_distancias'])  # Asumiendo que no hay valores 0 en esta columna
'''
# Normalizar los datos
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Aplicar K-Means con 2 clusters
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(features_scaled)

# Asignar los clusters a tu DataFrame
df_flag['cluster_risk'] = clusters

# Puedes renombrar los clusters como 'Riesgo' y 'Sin Riesgo' si lo prefieres
df_flag['cluster_risk'] = df_flag['cluster_risk'].map({0: 'Riesgo', 1: 'Sin Riesgo'})


In [67]:
df_flag.cluster_risk.value_counts()

Riesgo        17062
Sin Riesgo     5174
Name: cluster_risk, dtype: int64

In [64]:
df_flag.head()

,COD_SNIP,porc_incremento_monto,porc_gastado,porc_tiempo_adicional,ANO_ADJ,nro_proveedores,porc_sin_actividad,porc_sin_no_habido,promedio_distancias,porc_proveedor_cercano,cluster_risk
0,2535995,0.068550,0.023734,32.608696,2021,4,0.0,0.0,14.954779,100.0,Riesgo
1,2509842,0.460870,0.004433,97.802198,2021,3,0.0,0.0,21.216312,100.0,Riesgo
2,2526990,-0.000003,0.000000,4.444444,2021,3,0.0,0.0,21.216312,100.0,Riesgo
3,2527961,-0.000520,0.000000,29.032258,2021,3,0.0,0.0,21.216312,100.0,Riesgo
4,2535979,0.532620,0.022061,432.352941,2021,15,0.0,0.0,4.424103,100.0,Riesgo


In [53]:
df_proveedor.head()

,CUI,nro_proveedores,porc_sin_actividad,porc_sin_no_habido,promedio_distancias,porc_proveedor_cercano
0,2097656,3,0.000000,0.000000,10.034000,100.000000
1,2180790,81,1.234568,0.000000,36.193542,88.888889
2,2186811,1,0.000000,0.000000,0.000000,100.000000
3,2190175,30,3.333333,3.333333,109.747860,56.666667
4,2191823,5,0.000000,0.000000,120.315794,40.000000


In [ ]:
    # Aplicar K-means para identificar 4 clusters
    kmeans = KMeans(n_clusters=4, random_state=0).fit(data_df[['suma_ponderada']])
    data_df['categoria_cluster'] = kmeans.labels_

    # Ordenar las categorías y asignar nombres
    orden_categorias = data_df.groupby('categoria_cluster')['suma_ponderada'].mean().sort_values().index
    mapeo_orden = {v: i for i, v in enumerate(orden_categorias)}
    data_df['categoria_cluster'] = data_df['categoria_cluster'].map(mapeo_orden)
    nombres_categorias = ['Previo al inicio', 'En inicio', 'En proceso', 'Satisfactorio']
    data_df['categoria'] = data_df['categoria_cluster'].map(dict(enumerate(nombres_categorias)))

In [42]:
df_proveedor.head()

,CUI,RUC,latitude_proy,longitude_proy,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,LATITUDE,LONGITUDE,distancia_proveedor,flag_distancia
0,2362485,10149763302,-9.048611,-77.804722,ACTIVO,HABIDO,-12.062500,-77.129167,343.125480,1
1,2362485,10125278610,-9.048611,-77.804722,ACTIVO,HABIDO,-12.029722,-77.010000,342.636215,1
2,2362485,20103368819,-9.048611,-77.804722,ACTIVO,HABIDO,-11.866667,-77.076944,323.264131,1
3,2362485,10105450160,-9.048611,-77.804722,ACTIVO,HABIDO,-5.152500,-80.657778,535.335425,1
4,2362485,10124930450,-9.048611,-77.804722,ACTIVO,HABIDO,-12.075556,-76.993611,348.016825,1


In [46]:
df_proveedor.CONDICION_PROVEEDOR.value_counts()

HABIDO           324702
NO HALLADO         1384
PENDIENTE           318
NO HABIDO           174
POR VERIFICAR        15
Name: CONDICION_PROVEEDOR, dtype: int64

In [47]:
df_proveedor.ESTADO_PROVEEDOR.value_counts()

ACTIVO                       317070
SUSPENSION TEMPORAL            7784
BAJA PROV. POR OFICIO          1630
BAJA DEFINITIVA                  46
INHABILITADO-VENT.UNICA          34
BAJA DE OFICIO                   24
BAJA PROVISIONAL                  3
PENDIENTE DE INI. DE ACT.         3
Name: ESTADO_PROVEEDOR, dtype: int64

In [44]:
df_proveedor.loc[df_proveedor.flag_distancia==0]

,CUI,RUC,latitude_proy,longitude_proy,ESTADO_PROVEEDOR,CONDICION_PROVEEDOR,LATITUDE,LONGITUDE,distancia_proveedor,flag_distancia
22,2362485,20102948951,-9.048611,-77.804722,ACTIVO,HABIDO,-9.128611,-78.530833,80.220387,0
23,2499818,20102948951,-8.760278,-78.061389,ACTIVO,HABIDO,-9.128611,-78.530833,65.851095,0
30,2487112,20100006516,NaN,NaN,ACTIVO,HABIDO,-12.097778,-77.027222,NaN,0
40,2480490,20100006516,NaN,NaN,ACTIVO,HABIDO,-12.097778,-77.027222,NaN,0
46,2522919,20100006516,-12.042222,-77.026944,ACTIVO,HABIDO,-12.097778,-77.027222,6.177570,0
...,...,...,...,...,...,...,...,...,...,...
326628,2513084,10158564793,-5.941389,-77.977222,ACTIVO,HABIDO,-6.229444,-77.872778,34.048497,0
326629,2499384,10158508423,-5.735278,-77.925000,ACTIVO,HABIDO,-5.735278,-77.925000,0.000000,0
326630,2499011,20100432487,-8.188333,-76.509444,ACTIVO,HABIDO,-8.188333,-76.509444,0.000000,0
326631,2548755,20100851010,-12.048333,-77.000556,ACTIVO,HABIDO,-12.075556,-77.043333,5.549816,0


In [43]:
df_proveedor.flag_distancia.value_counts()

0    239706
1     86927
Name: flag_distancia, dtype: int64